# To Do
- eliminar as colunas claramente irrelevantes para a afinidade

In [ ]:
import pandas as pd
import numpy as np
import json

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_items', None)

## Filtrando colunas

In [ ]:
df_raw = pd.read_csv("../dados/inputs/microdados_ed_basica_2024.csv", sep=";", encoding="latin1")

print(df_raw.shape)

# é  um df muito grande (215_545, 426), primeiro passo: filtrar as colunas

In [ ]:
df_filtro = pd.read_excel("../dados/temporarios/filtro_colunas.xlsx", skiprows=8, usecols=[1, 2])

df_filtro = df_filtro[~df_filtro["util"].isin(["DESCARTAR", "TALVEZ", "AUSENTE", np.nan])]

lista_cols = df_filtro["Nome da Variável"].astype(str).to_list()

print(f"Reduzimos para {len(lista_cols)} colunas")

with open("../dados/banco_dados/colunas_relevantes_md_edb.json", "w", encoding="utf-8") as f:
    json.dump(lista_cols, f, ensure_ascii=False, indent=4)
    

In [ ]:

with open("../dados/banco_dados/colunas_relevantes_md_edb.json", "r", encoding="utf-8") as f:
    cols_interesse = json.load(f)

df_filtrado = df_raw[cols_interesse]

df_filtrado.shape


## Combinando colunas

In [ ]:
def soma_colunas(df: pd.DataFrame, colunas: list[str], novo_nome: str, drop = True):
    """
    df: DataFrame
    colunas: lista com nomes de colunas a serem somadas
    novo_nome: nome da nova coluna criada
    """
    df[novo_nome] = df[colunas].sum(axis=1).astype(int)
    if drop:
        df = df.drop(columns=colunas)
        
    return df



In [ ]:

df = df_filtrado.copy()


infraestrutura = ["IN_ESGOTO_REDE_PUBLICA", "IN_ENERGIA_REDE_PUBLICA",
                  "IN_ENERGIA_RENOVAVEL", "IN_LIXO_SERVICO_COLETA"]
df = soma_colunas(df, infraestrutura, "infraestrutura")



estrutura_pobre = ["IN_TERREIRAO", "IN_ACESSIBILIDADE_INEXISTENTE"]
df = soma_colunas(df, estrutura_pobre, "estrutura_pobre")

estrutura_basica = ["IN_ALMOXARIFADO", "IN_BIBLIOTECA", "IN_LABORATORIO_INFORMATICA",
                    "IN_PATIO_DESCOBERTO", "IN_QUADRA_ESPORTES_DESCOBERTA"]
df = soma_colunas(df, estrutura_basica, "estrutura_basica")

estrutura_padrao = ["IN_AREA_VERDE", "IN_AUDITORIO", "IN_BIBLIOTECA_SALA_LEITURA",
                  "IN_LABORATORIO_CIENCIAS", "IN_LABORATORIO_EDUC_PROF",
                  "IN_PATIO_COBERTO", "IN_SALA_MULTIUSO", "IN_SALA_LEITURA",]
df = soma_colunas(df, estrutura_padrao, "estrutura_padrao")

estrutura_premium = ["IN_AREA_PLANTIO", "IN_BANHEIRO_CHUVEIRO", "IN_PISCINA", "IN_SALA_ATELIE_ARTES",
                     "IN_SALA_MUSICA_CORAL", "IN_SALA_ESTUDIO_DANCA", "IN_SALA_ESTUDIO_GRAVACAO",
                     "IN_SALA_REPOUSO_ALUNO", "IN_ACESSIBILIDADE_ELEVADOR", "IN_MATERIAL_PED_MUSICAL"]
df = soma_colunas(df, estrutura_premium, "estrutura_premium")



ativos_basico = ["QT_EQUIP_TV", "QT_EQUIP_MULTIMIDIA", "QT_DESKTOP_ALUNO"]
df = soma_colunas(df, ativos_basico, "ativos_basico")

ativos_premium = ["QT_SALAS_UTILIZA_CLIMATIZADAS", "QT_EQUIP_LOUSA_DIGITAL",
                  "QT_COMP_PORTATIL_ALUNO", "QT_TABLET_ALUNO"]
df = soma_colunas(df, ativos_premium, "qt_ativos")



pessoal_basico = ["QT_PROF_ADMINISTRATIVOS", "QT_PROF_SERVICOS_GERAIS",
                  "QT_PROF_BIBLIOTECARIO", "QT_PROF_ALIMENTACAO", "QT_PROF_SECRETARIO"]
df = soma_colunas(df, pessoal_basico, "pessoal_basico")

pessoal_padrao = ["QT_PROF_SAUDE", "QT_PROF_COORDENADOR", "QT_PROF_PEDAGOGIA", "QT_PROF_MONITORES",
                "QT_PROF_GESTAO", "QT_PROF_ASSIST_SOCIAL"]
df = soma_colunas(df, pessoal_padrao, "pessoal_padrao")

pessoal_premium = ["QT_PROF_FONAUDIOLOGO", "QT_PROF_NUTRICIONISTA", "QT_PROF_PSICOLOGO",
               "QT_PROF_SEGURANCA", "QT_PROF_AGRICOLA"]
df = soma_colunas(df, pessoal_premium, "pessoal_premium")



alunos = ["QT_MAT_INF", "QT_MAT_FUND_AI", "QT_MAT_FUND_AF", "QT_MAT_MED"]
df = soma_colunas(df, alunos, "qt_alunos", drop=False)


professores = ["QT_DOC_INF", "QT_DOC_FUND_AI", "QT_DOC_FUND_AF", "QT_DOC_MED"]
df_combinado = soma_colunas(df, professores, "qt_professores")


df_combinado.shape

In [ ]:
df_combinado.head(20)

# CO_ENTIDADE
# TP_DEPENDENCIA
# TP_CATEGORIA_ESCOLA_PRIVADA
# CO_CEP
# TP_SITUACAO_FUNCIONAMENTO

# IN_MEDIACAO_PRESENCIAL


## Salvando resultado

In [ ]:
df_combinado.to_csv("../dados/temporarios/01-md_edb_colunas.csv", index=False)